In [1]:
# %pip install cx_Oracle
# %pip install pandas
# %pip install pyarrow
# %pip install snowflake

In [2]:
import configparser
config = configparser.ConfigParser()

config.read('toad.cfg')
username = config['TOAD']['toad_user']
password = config['TOAD']['toad_password']

In [3]:
import cx_Oracle
import pandas as pd

# Establish the database connection
host_name = 'xdhfd17-scan1'
service_n = 'CDW_IMDSCVP.thehartford.com'
port_num = '1521'

dsn_tns = cx_Oracle.makedsn(host_name, port_num, service_name=service_n)
connection = cx_Oracle.connect(user=username, password=password, dsn=dsn_tns)
 
try:
    # Create a cursor object
    cursor = connection.cursor()
 
    # Define the query
    query = """SELECT CLM_ID, LOSS_EVNT_NUM, POL_ID, COV_TYP_DESC, LOSS_DT, CLM_TTL_INC_AMT,
    CASE WHEN COV_TYP_DESC IN (\'Glass\', \'Towing/Labor\') THEN CLM_TTL_INC_AMT ELSE 0 END AS GLASS_TOW_AMT
    FROM DRAGON_PRD.AGL_MASTER_M_MV WHERE LOSS_DT >= \'01Jul2024\'"""
    # AGL_MASTER_M_MV
 
    # Execute the query
    cursor.execute(query)
 
    # Fetch all the rows
    rows = cursor.fetchall()

    # Get the column names
    col_names = [row for row in cursor.description]
 
    # Convert the result to a pandas DataFrame
    df = pd.DataFrame(rows, columns=[name[0] for name in col_names])
 
    # Print the DataFrame
    print(df.head())
 
finally:
    # Close the cursor and connection
    cursor.close()
    connection.close()

       CLM_ID LOSS_EVNT_NUM       POL_ID               COV_TYP_DESC  \
0  Y3QAP88280  CA0020567521  41CSES18112  Property Damage Liability   
1  Y3QAL87951  CA0020560831  53UENOH9019    Bodily Injury Liability   
2  Y4QAP74555  PA0020681722  55100268651  Property Damage Liability   
3  Y3YAL02691  PA0020499733  55PHE958901              Bodily Injury   
4  Y3YMD08097  PA0020563893  55PHG598574              Comprehensive   

     LOSS_DT  CLM_TTL_INC_AMT  GLASS_TOW_AMT  
0 2024-08-12          4110.20            0.0  
1 2024-08-07          5000.00            0.0  
2 2024-10-17             0.00            0.0  
3 2024-07-06             0.00            0.0  
4 2024-08-11           459.99            0.0  


In [4]:
# df.to_csv('oracle_claims.csv')

In [5]:
cols = df.select_dtypes(include=['datetime64[ns]']).columns
for col in cols:
    df[col] = pd.to_datetime(df[col]).dt.date

In [6]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

snowflake_pass = config['SNOWFLAKE']['secret_passphrase']
df.reset_index(drop=True, inplace=True)

with open("/tech/appl/default/user/pa08042e/rsa_snow.p8", "rb") as key:
    private_key= serialization.load_pem_private_key(
        key.read(),
        password= snowflake_pass.encode(),
        backend=default_backend()
    )

pkb = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

ctx = snowflake.connector.connect(
    user='pa08042',
    account='hfsg_prod.us-east-1',
    private_key=pkb,
    database='user_db',
    schema='AD1_PA08042'
    )

success, nchunks, nrows, _ = write_pandas(ctx, df,'TEST_TABLE', table_type = 'transient', use_logical_type=True, overwrite=True)
print(success)

True
